In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import matthews_corrcoef, make_scorer
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Activation, BatchNormalization, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# Loading the Data
train_X = pd.read_csv('data/pca_train_data.csv', index_col=0)
metadata = pd.read_csv('data/select_train_meta.csv', index_col=0)
train_Y = metadata['target']

train_X = np.expand_dims(train_X, axis=2)
n_samples, n_features, n_outputs = train_X.shape
print(n_samples, n_features, n_outputs)

2625 369 1


In [4]:
learning_rate = 0.005
train_epochs=100

In [5]:
def cnn_model():
    # Define the network
    model = Sequential()
    model.add(Conv1D(128, kernel_size=11, input_shape=(369, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(4))

    model.add(Conv1D(384, kernel_size=11))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(4))

    model.add(Conv1D(1024, kernel_size=11))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(4))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='mean_squared_error',
                  optimizer=optimizers.SGD(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model

In [6]:
neural_network = KerasClassifier(build_fn=cnn_model, epochs=train_epochs, batch_size=32, verbose=1)

In [7]:
scoring = {'accuracy': 'accuracy', 'recall': 'recall', 
           'precision': 'precision', 'f1': 'f1', 'mcc': make_scorer(matthews_corrcoef)}

In [8]:
cv_results = cross_validate(neural_network, train_X, train_Y, scoring=scoring, return_train_score=True)

Instructions for updating:
Colocations handled automatically by placer.


/anaconda3/envs/py3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.cast instead.
Epoch 1/15
  64/1750 [>.............................] - ETA: 36s - loss: 0.2329 - acc: 0.5469

KeyboardInterrupt: 

In [ ]:
cv_results

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
cv_folds = list(range(1, len(cv_results['fit_time']) + 1))
ax1.plot(cv_folds, cv_results['test_accuracy'] * 100)
ax1.plot(cv_folds, cv_results['test_recall'] * 100)
ax1.plot(cv_folds, cv_results['test_precision'] * 100)
ax1.plot(cv_folds, cv_results['test_f1'] * 100)
ax1.plot(cv_folds, cv_results['test_mcc'] * 100)
ax1.set(xlabel='%', ylabel='test_accuracy', title='CV Results')
ax1.legend(['test_accuracy', 'test_recall', 'test_precision', 'test_f1', 'test_mcc']);